### Imports

In [1]:
import numpy as np 
import pandas as pd
import requests  
import pycountry 
from google.oauth2 import service_account
from googleapiclient.discovery import build
from functions import *
from conf.settings import *
import re
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Data Extraction and Casting

In [2]:
df_metrics_by_platform = get_named_range_as_dataframe(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, PLATFORM_NAMED_RANGE)
df_metrics_by_platform = convert_column_names_to_standard_english(df_metrics_by_platform)
df_metrics_by_platform = clean_and_cast_columns(df_metrics_by_platform)

### Analysis

In [31]:
df_metrics_by_platform.columns

Index(['ad_set_name', 'platform', 'result_rate', 'result_indicator', 'results',
       'reach', 'frequency', 'links_clicks', 'cpc_link', 'cpc_general',
       'cost_per_1000_people_reached', 'ctr', 'add_to_cart',
       'cost_per_add_to_cart', 'initiate_checkout',
       'cost_per_initiate_checkout', 'purchase', 'cost_per_purchase',
       'amount_spent', 'purchase_conversion_value', 'campaign_goal',
       'classified_campaign_goal', 'classified_result_indicator'],
      dtype='object')

In [39]:
25/1619

0.015441630636195183

In [37]:
#df_metrics_by_platform.query('classified_campaign_goal == "Engajamento/Visualização"').platform.value_counts()
df_metrics_by_platform.platform.value_counts().sum()

np.int64(1619)

In [15]:
df_metrics_by_platform.purchase.sum()

np.float64(9475.0)

In [16]:
df_metrics_by_platform.query('classified_campaign_goal == "Conversão" and platform =="Audience Network"').purchase.sum()

np.float64(1.0)

In [3]:
df_metrics_by_platform['campaign_goal'] = df_metrics_by_platform['ad_set_name'].apply(create_column_campaign_goal)
df_metrics_by_platform['classified_campaign_goal'] = df_metrics_by_platform['campaign_goal'].apply(classify_campaign)

In [4]:
df_metrics_by_platform['campaign_goal'] = df_metrics_by_platform['ad_set_name'].apply(create_column_campaign_goal)
df_metrics_by_platform['classified_campaign_goal'] = df_metrics_by_platform['campaign_goal'].apply(classify_campaign)
df_metrics_by_platform['classified_result_indicator'] = df_metrics_by_platform['result_indicator'].apply(lambda x: 'pixel_initiate_checkout' if 'fb_pixel_initiate_checkout' in x else  
                                                                                                         'pixel_add_to_cart' if 'fb_pixel_add_to_cart' in x else              
                                                                                                         'pixel_purchase' if 'fb_pixel_purchase' in x else                    
                                                                                                         'post_engagement' if 'post_engagement' in x else                     
                                                                                                         'Outros')

In [5]:
print(df_metrics_by_platform['classified_campaign_goal'].value_counts())
print(df_metrics_by_platform['platform'].value_counts())
print(df_metrics_by_platform['result_indicator'].value_counts())

classified_campaign_goal
Conversão                   1270
Engajamento/Visualização     338
Outros                        11
Name: count, dtype: int64
platform
Facebook            970
Instagram           521
Audience Network    128
Name: count, dtype: int64
result_indicator
actions:offsite_conversion.fb_pixel_add_to_cart          467
actions:offsite_conversion.fb_pixel_initiate_checkout    412
actions:offsite_conversion                               271
-                                                        258
actions:offsite_conversion.fb_pixel_purchase             174
actions:post_engagement                                   28
actions:link_click                                         5
estimated_ad_recallers                                     4
Name: count, dtype: int64


In [6]:
platform_grouped = (df_metrics_by_platform.groupby(["platform", "classified_campaign_goal", "classified_result_indicator"]).agg({"results": "sum",                      
                                                                                                                  "result_rate": "mean",      
                                                                                                                  "reach": "sum",                     
                                                                                                                  "frequency": "mean",             
                                                                                                                  "cpc_link": "mean",                     
                                                                                                                  "cpc_general": "mean",                   
                                                                                                                  "cost_per_1000_people_reached": "mean",       
                                                                                                                  "ctr": "mean",                          
                                                                                                                  "add_to_cart": "sum",            
                                                                                                                  "cost_per_add_to_cart": "mean",           
                                                                                                                  "initiate_checkout": "sum",                    
                                                                                                                  "cost_per_initiate_checkout": "mean",        
                                                                                                                  "purchase": "sum",                 
                                                                                                                  "cost_per_purchase": "mean",             
                                                                                                                  "amount_spent": "sum",                       
                                                                                                                  "purchase_conversion_value": "sum",
                                                                                                                  "links_clicks": "sum",       
                                                                                                                  }).reset_index()
                                                                                                                  .rename(columns={'ad_set_name': 'qty_ad_sets'}))

platform_grouped["roi"] = platform_grouped["purchase_conversion_value"] / platform_grouped["amount_spent"]
platform_grouped["cart_to_checkout_conversion"] = np.where(platform_grouped["add_to_cart"] > 0, 
                                                      platform_grouped["initiate_checkout"] / platform_grouped["add_to_cart"], 
                                                      0)
platform_grouped["checkout_to_purchase_conversion"] = np.where(platform_grouped["initiate_checkout"] > 0,  
                                                          platform_grouped["purchase"] / platform_grouped["initiate_checkout"], 
                                                          0)

In [7]:
platform_grouped.head()

,platform,classified_campaign_goal,classified_result_indicator,results,result_rate,reach,frequency,cpc_link,cpc_general,cost_per_1000_people_reached,ctr,add_to_cart,cost_per_add_to_cart,initiate_checkout,cost_per_initiate_checkout,purchase,cost_per_purchase,amount_spent,purchase_conversion_value,links_clicks,roi,cart_to_checkout_conversion,checkout_to_purchase_conversion
0,Audience Network,Conversão,Outros,0,0.000000,282687.0,2.086500,0.01750,0.017000,0.64700,0.019520,49,0.476000,3,1.121500,0.0,0.000000,132.47,0.00,13553,0.000000,0.061224,0.000000
1,Audience Network,Conversão,pixel_add_to_cart,23,0.000500,31059.0,1.790000,0.03500,0.035000,0.60500,0.010900,23,3.100000,1,2.815000,0.0,0.000000,18.27,0.00,568,0.000000,0.043478,0.000000
2,Audience Network,Conversão,pixel_purchase,1,0.000000,39123.0,3.600000,0.01000,0.010000,0.76000,0.015700,24,1.240000,2,14.880000,1.0,29.750000,29.75,19.71,2176,0.662521,0.083333,0.500000
3,Audience Network,Engajamento/Visualização,Outros,8141,0.011779,109655.0,4.748447,0.02699,0.026796,2.48301,0.018318,648,0.633981,155,0.851553,52.0,0.964175,357.21,1792.05,12168,5.016797,0.239198,0.335484
4,Audience Network,Outros,Outros,0,0.000000,2235.0,1.355000,0.02500,0.025000,1.43000,0.035850,0,0.000000,0,0.000000,0.0,0.000000,4.30,0.00,108,0.000000,0.000000,0.000000


In [8]:
print("Aggregated Metrics by platform Group:")
platform_grouped = convert_column_standard_english_to_portuguse(platform_grouped, base='EN')

Aggregated Metrics by platform Group:


### Save to sheets

In [9]:
column_order = ["plataforma",
                "objetivo_classificado_da_campanha",
                "classified_result_indicator",
                "resultados",
                "taxa_de_resultado",
                "valor_gasto",
                "alcance",
                "compra",
                "valor_conversao_compra",
                "roi",
                "frequencia",
                "custo_por_1000_pessoas_alcancadas",
                "cliques_no_link",
                "custo_por_clique_no_link",
                "custo_por_clique_geral",
                "ctr",
                "adicionar_ao_carrinho",
                "custo_por_adicionar_ao_carrinho",
                "iniciar_finalizacao_da_compra",
                "custo_por_iniciar_finalizacao_da_compra",
                "custo_por_compra",
                "conversao_carrinho_checkout",
                "conversao_checkout_compra",]
platform_grouped = platform_grouped[column_order]
platform_grouped = convert_column_standard_english_to_portuguse(platform_grouped, base = 'PT')
#save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_w_metrics_by_platform', platform_grouped)


{'spreadsheetId': '1HVlgRHzBjLS4yO9bOKrTeJwlitn9u40CU1rSFwH4VZo',
 'updatedRange': "'Analytics PLATFORM'!B2:X23",
 'updatedRows': 22,
 'updatedColumns': 23,
 'updatedCells': 506}

In [10]:
platform_grouped

,platform,classified_campaign_goal,classified_result_indicator,results,result_rate,amount_spent,reach,purchase,purchase_conversion_value,roi,frequency,cost_per_1000_people_reached,links_clicks,cpc_link,cpc_general,ctr,add_to_cart,cost_per_add_to_cart,initiate_checkout,cost_per_initiate_checkout,cost_per_purchase,cart_to_checkout_conversion,checkout_to_purchase_conversion
0,Audience Network,Conversão,Outros,0,0.000000,132.47,282687.0,0.0,0.00,0.000000,2.086500,0.647000,13553,0.017500,0.017000,0.019520,49,0.476000,3,1.121500,0.000000,0.061224,0.000000
1,Audience Network,Conversão,pixel_add_to_cart,23,0.000500,18.27,31059.0,0.0,0.00,0.000000,1.790000,0.605000,568,0.035000,0.035000,0.010900,23,3.100000,1,2.815000,0.000000,0.043478,0.000000
2,Audience Network,Conversão,pixel_purchase,1,0.000000,29.75,39123.0,1.0,19.71,0.662521,3.600000,0.760000,2176,0.010000,0.010000,0.015700,24,1.240000,2,14.880000,29.750000,0.083333,0.500000
3,Audience Network,Engajamento/Visualização,Outros,8141,0.011779,357.21,109655.0,52.0,1792.05,5.016797,4.748447,2.483010,12168,0.026990,0.026796,0.018318,648,0.633981,155,0.851553,0.964175,0.239198,0.335484
4,Audience Network,Outros,Outros,0,0.000000,4.30,2235.0,0.0,0.00,0.000000,1.355000,1.430000,108,0.025000,0.025000,0.035850,0,0.000000,0,0.000000,0.000000,0.000000,0.000000
5,Facebook,Conversão,Outros,4,0.000323,2152.28,705795.0,0.0,0.00,0.000000,1.099231,3.262051,3459,0.566838,0.171026,0.016327,593,3.009402,79,6.068376,0.000000,0.133221,0.000000
6,Facebook,Conversão,pixel_add_to_cart,39092,0.001297,42376.98,18430546.0,1672.0,53201.18,1.255426,1.418322,3.106364,242948,0.396434,0.098252,0.026493,39092,3.019231,7270,9.466923,24.318881,0.185972,0.229986
7,Facebook,Conversão,pixel_initiate_checkout,14497,0.000257,68045.70,33075549.0,3386.0,105497.86,1.550397,1.451854,2.658046,380338,2.331689,0.085099,0.026104,63723,2.372550,14497,9.805695,27.824967,0.227500,0.233566
8,Facebook,Conversão,pixel_purchase,1993,0.000093,44430.39,14889413.0,1993.0,64826.41,1.459056,1.420759,3.312552,195723,0.460069,0.137724,0.022095,36607,2.498276,8088,10.822414,32.177034,0.220941,0.246414
9,Facebook,Engajamento/Visualização,Outros,66525,0.043132,7441.44,427242.0,706.0,24203.99,3.252595,5.075728,27.097767,17481,0.675825,0.407670,0.017703,11204,1.017476,2735,4.492136,12.290583,0.244109,0.258135


### Métricas de conversão e engajamento

In [11]:
df_metricas_by_platform_engajamento = calcular_percentuais_por_objetivo(df_metrics_by_platform, 'platform', 'Engajamento/Visualização')
#save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_analysis_by_platform_engagement', df_metricas_by_platform_engajamento)
df_metricas_by_platform_engajamento

,platform,classified_campaign_goal,results,purchase,purchase_conversion_value,amount_spent,count,pct_results,pct_purchase,pct_purchase_conversion_value,pct_amount_spent,pct_count
0,Audience Network,Engajamento/Visualização,8141,52.0,1792.05,357.21,103,0.0,0.05,0.05,0.03,0.3
1,Facebook,Engajamento/Visualização,636568,716.0,24468.04,7961.15,116,0.21,0.73,0.73,0.69,0.34
2,Instagram,Engajamento/Visualização,2331743,218.0,7422.69,3173.47,119,0.78,0.22,0.22,0.28,0.35


In [ ]:
df_metricas_by_platform_conversao = calcular_percentuais_por_objetivo(df_metrics_by_platform, 'platform')
#save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_analysis_by_platform_conversion', df_metricas_by_platform_conversao)
df_metricas_by_platform_conversao

,platform,classified_campaign_goal,results,purchase,purchase_conversion_value,amount_spent,count,pct_results,pct_purchase,pct_purchase_conversion_value,pct_amount_spent,pct_count
0,Audience Network,Conversão,24,1.0,19.71,180.49,23,0.0,0.00,0.00,0.00,0.02
1,Facebook,Conversão,55586,7051.0,223525.45,157005.35,850,0.71,0.83,0.83,0.78,0.67
2,Instagram,Conversão,22827,1425.0,46829.84,43243.41,397,0.29,0.17,0.17,0.22,0.31


In [13]:
df_metricas_by_platform_conversao.dtypes

platform                          object
classified_campaign_goal          object
results                            Int64
purchase                         float64
purchase_conversion_value        float64
amount_spent                     float64
count                              Int64
pct_results                      Float64
pct_purchase                     float64
pct_purchase_conversion_value    float64
pct_amount_spent                 float64
pct_count                        Float64
dtype: object